In [9]:
pip install huggingface_hub[hf_xet]

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install PyPDF2

In [2]:
!pip install transformers accelerate bitsandbytes sentence-transformers faiss-cpu

In [3]:
import os
import pickle
import faiss
from sentence_transformers import SentenceTransformer

# Define filenames
INDEX_FILE = "faiss_index.idx"
SOURCE_FILE = "sources.pkl"

if os.path.exists(INDEX_FILE) and os.path.exists(SOURCE_FILE):
    print("Loading existing FAISS index and sources...")
    index = faiss.read_index(INDEX_FILE)
    with open(SOURCE_FILE, "rb") as f:
        sources = pickle.load(f)
else:
    print("Index not found. Creating new FAISS index from PDFs...")

Index not found. Creating new FAISS index from PDFs...


In [4]:
def simple_chunk_text(text, chunk_size=500, chunk_overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks


folder_path = r"C:\Users\pushk\Desktop\chatbot\DocumentDatasetMultiple"

def load_and_chunk_pdfs(folder_path):
    import os
    from PyPDF2 import PdfReader

    chunks = []
    sources = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            full_path = os.path.join(folder_path, filename)
            with open(full_path, "rb") as f:
                reader = PdfReader(f)
                raw_text = ""
                for page in reader.pages:
                    raw_text += page.extract_text() or ""
                split_texts = simple_chunk_text(raw_text)
                chunks.extend(split_texts)
                sources.extend([filename] * len(split_texts))
    
    return chunks, sources

chunks, sources = load_and_chunk_pdfs(folder_path)


In [5]:
from sentence_transformers import SentenceTransformer

# Load model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Convert chunks to embeddings
embeddings = embedder.encode(chunks, batch_size=8, convert_to_numpy=True)


In [6]:
import faiss
import numpy as np

# Convert to float32 (required by FAISS)
embeddings = np.array(embeddings).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [7]:
def search(query, top_k=5):
    query_embedding = embedder.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    results = [chunks[i] for i in indices[0]]
    return results


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True).eval()


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

C:\Users\pushk\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pushk\.cache\huggingface\hub\models--Qwen--Qwen3-0.6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype="auto"
)


In [22]:
# def generate_response(prompt, max_tokens=256):
#     inputs = tokenizer(prompt, return_tensors="pt")
#     outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)



# def chat_with_qwen(messages, max_tokens=512):
#     # messages: list of dicts like [{"role": "user", "content": "Hi"}]
#     prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     inputs = tokenizer(prompt, return_tensors="pt")
#     outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)


# import re

# def clean_qwen_response(raw_output):
#     # Strip <think> tags and anything inside them
#     cleaned = re.sub(r"<think>.*?</think>", "", raw_output, flags=re.DOTALL)
#     return cleaned.strip()

def chat_with_qwen(user_input, history=[]):
    # Build conversation history into Qwen chat format
    messages = []
    for role, content in history:
        if role == "user":
            messages.append(f"<|im_start|>user\n{content}<|im_end|>")
        elif role == "assistant":
            messages.append(f"<|im_start|>assistant\n{content}<|im_end|>")
    # Add the new user message
    messages.append(f"<|im_start|>user\n{user_input}<|im_end|>\n<|im_start|>assistant")

    # Join full prompt
    prompt = "\n".join(messages)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

    # Extract only the new text generated
    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)
    return output_text


In [14]:
# user_input = "What are the prerequisites of the subject Cyber Security?"
# response = generate_response(user_input)
# print(response)


chat_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What are the prerequisites of the subject Cyber Security?"}
]

In [15]:
response_raw = chat_with_qwen(chat_history)
response = clean_qwen_response(response_raw)
print(response)


# def chat_with_qwen(user_input, history=[]):
#     # Add your code here to call Qwen with streaming/history if needed
#     raw_response = your_qwen_generate_function(user_input, history)
#     return raw_response


system
You are a helpful assistant.
user
What are the prerequisites of the subject Cyber Security?
assistant


The prerequisites for a Cyber Security subject typically include the following core areas:

1. **Computer Science and Mathematics**: A solid foundation in computer science principles, programming (e.g., Python, C++, or Java), and basic mathematics (e.g., algebra, calculus) are essential.  
2. **Computer Literacy**: Familiarity with operating systems, software applications, and basic data handling.  
3. **Data Analysis and Programming**: Proficiency in programming and data analysis tools like Python, SQL, or R.  
4. **Ethics and Cybersecurity**: Understanding of cybersecurity principles, ethical considerations, and the impact of cyber threats.  

Some programs may also require a bachelor’s degree in computer science or a related field, but these are usually part of the degree requirements rather than the subject-specific prerequisites. For specific programs, consult the institu

In [17]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/13/e6/69fcbae3dd2fcb2f54283a7cbe03c8b944b79997f1b526984f91d4796a02/streamlit-1.45.1-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.5.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/72/76/20fa66124dbe6be5cafeb312ece67de6b61dd91a0247d1ea13db4ebb33c2/cachetools-5.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<7,>=3.20 from https://files.pythonhosted.org/packages/e4/07/cc9b0cbf7593f6ef8cf87fa9b0e55cd74c5cb526dd89ad84aa7d6547ef8d/

In [23]:
# import streamlit as st
# import re

# # Import your Qwen chatbot function
# # Replace this with your actual chatbot call
# def chat_with_qwen(user_input, history=[]):
#     # Dummy response for illustration; use your real Qwen API/chat call
#     raw_response = f"<think>Thinking...</think>\nThe answer to your question is:\n1. Point A\n2. Point B"
#     return raw_response

# # Clean Qwen output
# def clean_qwen_response(raw_output):
#     return re.sub(r"<think>.*?</think>", "", raw_output, flags=re.DOTALL).strip()

# # Streamlit app
# st.set_page_config(page_title="Qwen Chatbot", layout="centered")
# st.title("🧠 Qwen Chatbot Interface")

# # Store chat history
# if "chat_history" not in st.session_state:
#     st.session_state.chat_history = []

# # User input
# user_input = st.chat_input("Ask me anything...")

# if user_input:
#     st.session_state.chat_history.append(("user", user_input))

#     # Call the Qwen chatbot
#     raw_response = chat_with_qwen(user_input, st.session_state.chat_history)
#     cleaned_response = clean_qwen_response(raw_response)

#     st.session_state.chat_history.append(("assistant", cleaned_response))

# # Display chat history
# for sender, msg in st.session_state.chat_history:
#     if sender == "user":
#         st.chat_message("user").markdown(msg)
#     else:
#         st.chat_message("assistant").markdown(msg)


import streamlit as st
import re

def clean_qwen_response(raw_output):
    return re.sub(r"<think>.*?</think>", "", raw_output, flags=re.DOTALL).strip()

# Streamlit app UI
st.set_page_config(page_title="Qwen Chatbot", layout="centered")
st.title("🧠 Qwen Chatbot Interface")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.chat_input("Ask me anything...")

if user_input:
    st.session_state.chat_history.append(("user", user_input))
    raw_response = chat_with_qwen(user_input, st.session_state.chat_history)
    cleaned_response = clean_qwen_response(raw_response)
    st.session_state.chat_history.append(("assistant", cleaned_response))

for sender, msg in st.session_state.chat_history:
    if sender == "user":
        st.chat_message("user").markdown(msg)
    else:
        st.chat_message("assistant").markdown(msg)


2025-05-25 11:40:00.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 11:40:00.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Usage: streamlit run [OPTIONS] TARGET [ARGS]...

  Run a Python script, piping stderr to Streamlit.

  The script can be local or it can be an url. In the latter case, Streamlit
  will download the script to a temporary file and runs this file.

Options:
  --global.disableWidgetStateDuplicationWarning BOOLEAN
                                  By default, Streamlit displays a warning
                                  when a user sets both a widget default value
                                  in the function defining the widget and a
                                  widget value via the widget's key in
                                  `st.session_state`.
                                  
                                  If you'd like to turn off this warning, set
                                  this to True.  [env var: STREAMLIT_GLOBAL_DI
                                  SABLE_WIDGET_STATE_DUPLICATION_WARNING]
  --global.showWarningOnDirectExecution BOOLEAN
                    